In [1]:
import numpy as np
import pandas as pd

from folktables import ACSDataSource,ACSIncome


In [2]:
# download data
data_source = ACSDataSource(survey_year='2014', horizon='1-Year', survey='person') #you can change the survey year from 2014-2018
data = data_source.get_data(download=True)
features, labels, _ = ACSIncome.df_to_numpy(data)


In [3]:
data.head(3)

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
0,P,2017000000016,6,1,2500,3,1,1011189,206,73,...,60,46,260,300,256,370,366,293,250,62
1,P,2017000000031,6,1,1800,3,1,1011189,45,31,...,55,31,11,19,43,103,14,49,49,98
2,P,2017000000061,6,1,2400,3,1,1011189,136,41,...,101,47,160,333,176,33,131,74,30,185


In [4]:
# extract features
features = ['AGEP','COW','SCHL','MAR','OCCP','POBP','RELP','WKHP','SEX','RAC1P','PINCP']
adult_folk = data[features]
adult_folk.shape

(3204447, 11)

In [5]:
adult_folk.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,PINCP
0,73,NaN,10.0,4,NaN,1,0,NaN,1,2,10000.0
1,31,1.0,21.0,5,350.0,13,0,42.0,2,1,38500.0
2,41,1.0,17.0,1,6260.0,13,0,42.0,1,1,82000.0
3,48,NaN,16.0,1,NaN,17,1,NaN,2,1,8700.0
4,16,NaN,13.0,5,NaN,1,2,NaN,1,1,0.0


In [6]:
# rename target feature
adult_folk = adult_folk.rename(columns={'PINCP':"observed"})
adult_folk.head(2)

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,observed
0,73,NaN,10.0,4,NaN,1,0,NaN,1,2,10000.0
1,31,1.0,21.0,5,350.0,13,0,42.0,2,1,38500.0


In [7]:
adult_folk.shape

(3204447, 11)

In [8]:
# remove nans
adult_folk.dropna(inplace=True)
adult_folk.shape

(1656232, 11)

In [9]:
# convert observed to binary (currently using 50000 as the threshold)
adult_folk['observed'] = (adult_folk['observed'] >= 50000).astype(int)
adult_folk.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,observed
1,31,1.0,21.0,5,350.0,13,0,42.0,2,1,0
2,41,1.0,17.0,1,6260.0,13,0,42.0,1,1,1
5,37,4.0,16.0,3,230.0,1,0,30.0,2,2,0
10,32,7.0,16.0,1,8140.0,303,0,60.0,1,1,1
11,54,1.0,20.0,1,136.0,1,1,40.0,2,2,1


In [10]:
# helper functions to bin continuous features
def bin_age(age):
    if age <= 5:
        return 'toddler'
    elif age <= 12:
        return 'child'
    elif age <= 19:
        return 'teen'
    elif age <= 30:
        return '20 - 30'
    elif age <= 50:
        return '30 - 50'
    elif age <= 70:
        return '50 - 70'
    else:
        return '>70'

def bin_wh(hours):
    if hours <= 10:
        return '0 - 10'
    elif hours <= 20:
        return '10 - 20'
    elif hours <= 30:
        return '20 - 30'
    elif hours <= 40:
        return '30 - 40'
    elif hours <= 50:
        return '40 - 50'
    elif hours <= 60:
        return '50 - 60'
    elif hours <= 70:
        return '60 - 70'
    elif hours <= 80:
        return '70 - 80'
    elif hours <= 90:
        return '80 - 90'
    elif hours <= 100:
        return '90 - 99'
    else:
        return '>= 100'

In [11]:
# bin age
adult_folk['AGEP'] = adult_folk['AGEP'].apply(bin_age)

In [12]:
# bin work hours per week
adult_folk['WKHP'] = adult_folk.WKHP.apply(bin_wh)

In [13]:
adult_folk.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,observed
1,30 - 50,1.0,21.0,5,350.0,13,0,40 - 50,2,1,0
2,30 - 50,1.0,17.0,1,6260.0,13,0,40 - 50,1,1,1
5,30 - 50,4.0,16.0,3,230.0,1,0,20 - 30,2,2,0
10,30 - 50,7.0,16.0,1,8140.0,303,0,50 - 60,1,1,1
11,50 - 70,1.0,20.0,1,136.0,1,1,30 - 40,2,2,1


In [14]:
income = adult_folk.copy()

In [15]:
# map codes to string values for each columns
# map relationship status
income['RELP'] = income['RELP'].map({ 0: 'Reference person',
                                       1: 'Husband/wife',
                                       2: 'Biological son or daughter',
                                       3: 'Adopted son or daughter',
                                       4: 'Stepson or stepdaughter',
                                       5: 'Brother or sister',
                                       6: 'Father or mother',
                                       7: 'Grandchild',
                                       8: 'Parent-in-law',
                                       9: 'Son-in-law or daughter-in-law',
                                       10: 'Other relative',
                                       11: 'Roomer or boarder',
                                       12: 'Housemate or roommate',
                                       13: 'Unmarried partner',
                                       14: 'Foster child',
                                       15: 'Other nonrelative',
                                       16: 'Institutionalized group quarters population',
                                       17: 'Noninstitutionalized group quarters population'})

income.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,observed
1,30 - 50,1.0,21.0,5,350.0,13,Reference person,40 - 50,2,1,0
2,30 - 50,1.0,17.0,1,6260.0,13,Reference person,40 - 50,1,1,1
5,30 - 50,4.0,16.0,3,230.0,1,Reference person,20 - 30,2,2,0
10,30 - 50,7.0,16.0,1,8140.0,303,Reference person,50 - 60,1,1,1
11,50 - 70,1.0,20.0,1,136.0,1,Husband/wife,30 - 40,2,2,1


In [16]:
# map marital status
income['MAR'] = income['MAR'].map({ 1: 'Married', 2: 'Widowed', 3: 'Divorced', 4: 'Separated', 5: 'Never married or under 15 years old'})

income.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,observed
1,30 - 50,1.0,21.0,Never married or under 15 years old,350.0,13,Reference person,40 - 50,2,1,0
2,30 - 50,1.0,17.0,Married,6260.0,13,Reference person,40 - 50,1,1,1
5,30 - 50,4.0,16.0,Divorced,230.0,1,Reference person,20 - 30,2,2,0
10,30 - 50,7.0,16.0,Married,8140.0,303,Reference person,50 - 60,1,1,1
11,50 - 70,1.0,20.0,Married,136.0,1,Husband/wife,30 - 40,2,2,1


In [17]:
# map school
income['SCHL'] = income['SCHL'].map({ 
                                    1: 'No schooling completed',
                                    2: 'Nursery school/preschool',
                                    3: 'Kindergarten',
                                    4: 'Grade 1',
                                    5: 'Grade 2', 6: 'Grade 3',
                                    7: 'Grade 4', 8: 'Grade 5',
                                    9: 'Grade 6', 10: 'Grade 7',
                                    11: 'Grade 8', 12: 'Grade 9',
                                    13: 'Grade 10', 14: 'Grade 11',
                                    15: '12th Grade - no diploma',
                                    16: 'Regular high school diploma',
                                    17: 'GED or alternative credential',
                                    18: 'Some college but less than 1 year',
                                    19: '1 or more years of college credit but no degree',
                                    20: 'Associate’s degree',
                                    21: 'Bachelor’s degree',
                                    22: 'Master’s degree',
                                    23: 'Professional degree beyond a bachelor’s degree',
                                    24: 'Doctorate degree' })
income.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,observed
1,30 - 50,1.0,Bachelor’s degree,Never married or under 15 years old,350.0,13,Reference person,40 - 50,2,1,0
2,30 - 50,1.0,GED or alternative credential,Married,6260.0,13,Reference person,40 - 50,1,1,1
5,30 - 50,4.0,Regular high school diploma,Divorced,230.0,1,Reference person,20 - 30,2,2,0
10,30 - 50,7.0,Regular high school diploma,Married,8140.0,303,Reference person,50 - 60,1,1,1
11,50 - 70,1.0,Associate’s degree,Married,136.0,1,Husband/wife,30 - 40,2,2,1


In [18]:
# map class of worker(COW)
income['COW'] = income['COW'].map({
                       1: 'Employee of a private for-profit company or business, or of an individual, for wages,salary, or commissions',
                       2: 'Employee of a private not-for-profit, tax-exempt, or charitable organization',
                       3: 'Local government employee (city, county, etc.)',
                       4: 'State government employee',
                       5: 'Federal government employee',
                       6: 'Self-employed in own not incorporated business, professional practice, or farm',
                       7: 'Self-employed in own incorporated business, professional practice or farm',
                       8: 'Working without pay in family business or farm' })

income.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,observed
1,30 - 50,Employee of a private for-profit company or bu...,Bachelor’s degree,Never married or under 15 years old,350.0,13,Reference person,40 - 50,2,1,0
2,30 - 50,Employee of a private for-profit company or bu...,GED or alternative credential,Married,6260.0,13,Reference person,40 - 50,1,1,1
5,30 - 50,State government employee,Regular high school diploma,Divorced,230.0,1,Reference person,20 - 30,2,2,0
10,30 - 50,"Self-employed in own incorporated business, pr...",Regular high school diploma,Married,8140.0,303,Reference person,50 - 60,1,1,1
11,50 - 70,Employee of a private for-profit company or bu...,Associate’s degree,Married,136.0,1,Husband/wife,30 - 40,2,2,1


In [19]:
# map race
income['RAC1P'] = income['RAC1P'].map({ 1: 'White alone',
                                        2: 'Black or African American alone',
                                        3: 'American Indian alone',
                                        4: 'Alaska Native alone',
                                        5: 'American Indian and Alaska Native tribes specified, or American Indian or Alaska Native, not specified and no other races',
                                        6: 'Asian alone',
                                        7: 'Native Hawaiian and Other Pacific Islander alone',
                                        8: 'Some Other Race alone',
                                        9: 'Two or More Races'})

income.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,observed
1,30 - 50,Employee of a private for-profit company or bu...,Bachelor’s degree,Never married or under 15 years old,350.0,13,Reference person,40 - 50,2,White alone,0
2,30 - 50,Employee of a private for-profit company or bu...,GED or alternative credential,Married,6260.0,13,Reference person,40 - 50,1,White alone,1
5,30 - 50,State government employee,Regular high school diploma,Divorced,230.0,1,Reference person,20 - 30,2,Black or African American alone,0
10,30 - 50,"Self-employed in own incorporated business, pr...",Regular high school diploma,Married,8140.0,303,Reference person,50 - 60,1,White alone,1
11,50 - 70,Employee of a private for-profit company or bu...,Associate’s degree,Married,136.0,1,Husband/wife,30 - 40,2,Black or African American alone,1


In [20]:
# map sex
income['SEX'] = income['SEX'].map({ 1: 'Male', 2:'Female'})
income.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,observed
1,30 - 50,Employee of a private for-profit company or bu...,Bachelor’s degree,Never married or under 15 years old,350.0,13,Reference person,40 - 50,Female,White alone,0
2,30 - 50,Employee of a private for-profit company or bu...,GED or alternative credential,Married,6260.0,13,Reference person,40 - 50,Male,White alone,1
5,30 - 50,State government employee,Regular high school diploma,Divorced,230.0,1,Reference person,20 - 30,Female,Black or African American alone,0
10,30 - 50,"Self-employed in own incorporated business, pr...",Regular high school diploma,Married,8140.0,303,Reference person,50 - 60,Male,White alone,1
11,50 - 70,Employee of a private for-profit company or bu...,Associate’s degree,Married,136.0,1,Husband/wife,30 - 40,Female,Black or African American alone,1


In [21]:
# map POBP
# df = pd.read_excel('pob_codes.xlsx')
# state_mapping = dict(zip(list(df.code.values),list(df.state.values)))
income['POBP'] = income['POBP'].map({1.0: 'Alabama/AL',
 2.0: 'Alaska/AK',
 4.0: 'Arizona/AZ',
 5.0: 'Arkansas/AR',
 6.0: 'California/CA',
 8.0: 'Colorado/CO',
 9.0: 'Connecticut/CT',
 10.0: 'Delaware/DE',
 11.0: 'District of Columbia/DC',
 12.0: 'Florida/FL',
 13.0: 'Georgia/GA',
 15.0: 'Hawaii/HI',
 16.0: 'Idaho/ID',
 17.0: 'Illinois/IL',
 18.0: 'Indiana/IN',
 19.0: 'Iowa/IA',
 20.0: 'Kansas/KS',
 21.0: 'Kentucky/KY',
 22.0: 'Louisiana/LA',
 23.0: 'Maine/ME',
 24.0: 'Maryland/MD',
 25.0: 'Massachusetts/MA',
 26.0: 'Michigan/MI',
 27.0: 'Minnesota/MN',
 28.0: 'Mississippi/MS',
 29.0: 'Missouri/MO',
 30.0: 'Montana/MT',
 31.0: 'Nebraska/NE',
 32.0: 'Nevada/NV',
 33.0: 'New Hampshire/NH',
 34.0: 'New Jersey/NJ',
 35.0: 'New Mexico/NM',
 36.0: 'New York/NY',
 37.0: 'North Carolina/NC',
 38.0: 'North Dakota/ND',
 39.0: 'Ohio/OH',
 40.0: 'Oklahoma/OK',
 41.0: 'Oregon/OR',
 42.0: 'Pennsylvania/PA',
 44.0: 'Rhode Island/RI',
 45.0: 'South Carolina/SC',
 46.0: 'South Dakota/SD',
 47.0: 'Tennessee/TN',
 48.0: 'Texas/TX',
 49.0: 'Utah/UT',
 50.0: 'Vermont/VT',
 51.0: 'Virginia/VA',
 53.0: 'Washington/WA',
 54.0: 'West Virginia/WV',
 55.0: 'Wisconsin/WI',
 56.0: 'Wyoming/WY',
 60.0: 'American Samoa',
 66.0: 'Guam',
 69.0: 'Commonwealth of Northern Mariana Islands',
 72.0: 'Puerto Rico',
 78.0: 'U.S. Virgin Islands',
 100.0: 'Albania',
 102.0: 'Austria',
 103.0: 'Belgium',
 104.0: 'Bulgaria',
 105.0: 'Czechoslovakia',
 106.0: 'Denmark',
 108.0: 'Finland',
 109.0: 'France',
 110.0: 'Germany',
 116.0: 'Greece',
 117.0: 'Hungary',
 118.0: 'Iceland',
 119.0: 'Ireland',
 120.0: 'Italy',
 126.0: 'Netherlands',
 127.0: 'Norway',
 128.0: 'Poland',
 129.0: 'Portugal',
 130.0: 'Azores Islands',
 132.0: 'Romania',
 134.0: 'Spain',
 136.0: 'Sweden',
 137.0: 'Switzerland',
 138.0: 'United Kingdom, not specified',
 139.0: 'England',
 140.0: 'Scotland',
 147.0: 'Yugoslavia',
 148.0: 'Czech Republic',
 149.0: 'Slovakia',
 150.0: 'Bosnia and Herzegovina',
 151.0: 'Croatia',
 152.0: 'Macedonia',
 154.0: 'Serbia',
 156.0: 'Latvia',
 157.0: 'Lithuania',
 158.0: 'Armenia',
 159.0: 'Azerbaijan',
 160.0: 'Belarus',
 161.0: 'Georgia',
 162.0: 'Moldova',
 163.0: 'Russia',
 164.0: 'Ukraine',
 165.0: 'USSR',
 168.0: 'Montenegro',
 169.0: 'Other Europe, not specified',
 200.0: 'Afghanistan',
 202.0: 'Bangladesh',
 203.0: 'Bhutan',
 205.0: 'Myanmar',
 206.0: 'Cambodia',
 207.0: 'China',
 208.0: 'Cyprus',
 209.0: 'Hong Kong',
 210.0: 'India',
 211.0: 'Indonesia',
 212.0: 'Iran',
 213.0: 'Iraq',
 214.0: 'Israel',
 215.0: 'Japan',
 216.0: 'Jordan',
 217.0: 'Korea',
 218.0: 'Kazakhstan',
 222.0: 'Kuwait',
 223.0: 'Laos',
 224.0: 'Lebanon',
 226.0: 'Malaysia',
 229.0: 'Nepal',
 231.0: 'Pakistan',
 233.0: 'Philippines',
 235.0: 'Saudi Arabia',
 236.0: 'Singapore',
 238.0: 'Sri Lanka',
 239.0: 'Syria',
 240.0: 'Taiwan',
 242.0: 'Thailand',
 243.0: 'Turkey',
 245.0: 'United Arab Emirates',
 246.0: 'Uzbekistan',
 247.0: 'Vietnam',
 248.0: 'Yemen',
 249.0: 'Asia',
 253.0: 'South Central Asia, not specified',
 254.0: 'Other Asia, not specified',
 300.0: 'Bermuda',
 301.0: 'Canada',
 303.0: 'Mexico',
 310.0: 'Belize',
 311.0: 'Costa Rica',
 312.0: 'El Salvador',
 313.0: 'Guatemala',
 314.0: 'Honduras',
 315.0: 'Nicaragua',
 316.0: 'Panama',
 321.0: 'Antigua & Barbuda',
 323.0: 'Bahamas',
 324.0: 'Barbados',
 327.0: 'Cuba',
 328.0: 'Dominica',
 329.0: 'Dominican Republic',
 330.0: 'Grenada',
 332.0: 'Haiti',
 333.0: 'Jamaica',
 339.0: 'St. Lucia',
 340.0: 'St. Vincent & the Grenadines',
 341.0: 'Trinidad & Tobago',
 343.0: 'West Indies',
 344.0: 'Caribbean, Not Specified',
 360.0: 'Argentina',
 361.0: 'Bolivia',
 362.0: 'Brazil',
 363.0: 'Chile',
 364.0: 'Colombia',
 365.0: 'Ecuador',
 368.0: 'Guyana',
 369.0: 'Paraguay',
 370.0: 'Peru',
 372.0: 'Uruguay',
 373.0: 'Venezuela',
 374.0: 'South America',
 399.0: 'Americas, not specified',
 400.0: 'Algeria',
 407.0: 'Cameroon',
 408.0: 'Cabo Verde',
 412.0: 'Congo',
 414.0: 'Egypt',
 416.0: 'Ethiopia',
 417.0: 'Eritrea',
 420.0: 'Gambia',
 421.0: 'Ghana',
 423.0: 'Guinea',
 427.0: 'Kenya',
 429.0: 'Liberia',
 430.0: 'Libya',
 436.0: 'Morocco',
 440.0: 'Nigeria',
 444.0: 'Senegal',
 447.0: 'Sierra Leone',
 448.0: 'Somalia',
 449.0: 'South Africa',
 451.0: 'Sudan',
 453.0: 'Tanzania',
 454.0: 'Togo',
 457.0: 'Uganda',
 459.0: 'Democratic Republic of Congo (Zaire)',
 460.0: 'Zambia',
 461.0: 'Zimbabwe',
 462.0: 'Africa',
 463.0: 'Eastern Africa, not specified',
 464.0: 'Northern Africa, not specified',
 467.0: 'Western Africa, not specified',
 468.0: 'Other Africa, not specified',
 501.0: 'Australia',
 508.0: 'Fiji',
 511.0: 'Marshall Islands',
 512.0: 'Micronesia',
 515.0: 'New Zealand',
 523.0: 'Tonga',
 527.0: 'Samoa',
 554.0: 'Other U.S. Island Areas, Oceania, Not Specified, or at Sea'})
income.head()


,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,observed
1,30 - 50,Employee of a private for-profit company or bu...,Bachelor’s degree,Never married or under 15 years old,350.0,Georgia/GA,Reference person,40 - 50,Female,White alone,0
2,30 - 50,Employee of a private for-profit company or bu...,GED or alternative credential,Married,6260.0,Georgia/GA,Reference person,40 - 50,Male,White alone,1
5,30 - 50,State government employee,Regular high school diploma,Divorced,230.0,Alabama/AL,Reference person,20 - 30,Female,Black or African American alone,0
10,30 - 50,"Self-employed in own incorporated business, pr...",Regular high school diploma,Married,8140.0,Mexico,Reference person,50 - 60,Male,White alone,1
11,50 - 70,Employee of a private for-profit company or bu...,Associate’s degree,Married,136.0,Alabama/AL,Husband/wife,30 - 40,Female,Black or African American alone,1


In [23]:
# map occupation
income['OCCP'] = income['OCCP'].map({10.0: 'Chief executives',
 30.0: 'Legislators',
 20.0: 'General and operations managers',
 40.0: 'Advertising and promotions managers',
 51.0: 'Marketing managers',
 52.0: 'Sales managers',
 60.0: 'Public relations and fundraising managers',
 101.0: 'Administrative services managers',
 102.0: 'Facilities managers',
 110.0: 'Computer and information systems managers',
 120.0: 'Financial managers',
 135.0: 'Compensation and benefits managers',
 136.0: 'Human resources managers',
 137.0: 'Training and development managers',
 140.0: 'Industrial production managers',
 150.0: 'Purchasing managers',
 160.0: 'Transportation, storage, and distribution managers',
 205.0: 'Farmers, ranchers, and other agricultural managers',
 220.0: 'Construction managers',
 230.0: 'Education and childcare administrators ',
 300.0: 'Architectural and engineering managers',
 310.0: 'Food service managers',
 335.0: 'Entertainment and recreation managers',
 340.0: 'Lodging managers',
 350.0: 'Medical and health services managers',
 360.0: 'Natural sciences managers',
 410.0: 'Property, real estate, and community association managers',
 420.0: 'Social and community service managers',
 425.0: 'Emergency management directors',
 440.0: 'Other managers',
 325.0: 'Funeral home managers',
 400.0: 'Postmasters and mail superintendents',
 426.0: 'Personal service managers, all other',
 500.0: 'Agents and business managers of artists, performers, and athletes',
 510.0: 'Buyers and purchasing agents, farm products',
 520.0: 'Wholesale and retail buyers, except farm products',
 530.0: 'Purchasing agents, except wholesale, retail, and farm products',
 540.0: 'Claims adjusters, appraisers, examiners, and investigators',
 565.0: 'Compliance officers',
 600.0: 'Cost estimators',
 630.0: 'Human resources workers',
 640.0: 'Compensation, benefits, and job analysis specialists',
 650.0: 'Training and development specialists',
 700.0: 'Logisticians',
 705.0: 'Project management specialists',
 710.0: 'Management analysts',
 725.0: 'Meeting, convention, and event planners',
 726.0: 'Fundraisers',
 735.0: 'Market research analysts and marketing specialists',
 750.0: 'Business operations specialists, all other',
 800.0: 'Accountants and auditors',
 810.0: 'Property appraisers and assessors',
 820.0: 'Budget analysts',
 830.0: 'Credit analysts',
 845.0: 'Financial and investment analysts',
 850.0: 'Personal financial advisors',
 860.0: 'Insurance underwriters',
 900.0: 'Financial examiners',
 910.0: 'Credit counselors and loan officers',
 930.0: 'Tax examiners and collectors, and revenue agents',
 940.0: 'Tax preparers',
 960.0: 'Other financial specialists',
 1005.0: 'Computer and information research scientists',
 1006.0: 'Computer systems analysts',
 1007.0: 'Information security analysts',
 1010.0: 'Computer programmers',
 1021.0: 'Software developers',
 1022.0: 'Software quality assurance analysts and testers',
 1031.0: 'Web developers',
 1032.0: 'Web and digital interface designers',
 1050.0: 'Computer support specialists',
 1065.0: 'Database administrators and architects',
 1105.0: 'Network and computer systems administrators',
 1106.0: 'Computer network architects',
 1108.0: 'Computer occupations, all other',
 1200.0: 'Actuaries',
 1220.0: 'Operations research analysts',
 1240.0: 'Other mathematical science occupations',
 1210.0: 'Mathematicians',
 1230.0: 'Statisticians',
 1305.0: 'Architects, except landscape and naval',
 1306.0: 'Landscape architects',
 1310.0: 'Surveyors, cartographers, and photogrammetrists',
 1320.0: 'Aerospace engineers',
 1330.0: 'Agricultural engineers',
 1340.0: 'Bioengineers and biomedical engineers',
 1350.0: 'Chemical engineers',
 1360.0: 'Civil engineers',
 1400.0: 'Computer hardware engineers',
 1410.0: 'Electrical and electronics engineers',
 1420.0: 'Environmental engineers',
 1430.0: 'Industrial engineers, including health and safety',
 1440.0: 'Marine engineers and naval architects',
 1450.0: 'Materials engineers',
 1460.0: 'Mechanical engineers',
 1520.0: 'Petroleum, mining and geological engineers, including mining safety engineers',
 1500.0: 'Mining and geological engineers, including mining safety engineers',
 1530.0: 'Other engineers',
 1510.0: 'Nuclear engineers',
 1541.0: 'Architectural and civil drafters',
 1545.0: 'Other drafters',
 1551.0: 'Electrical and electronic engineering technologists and technicians',
 1555.0: 'Other engineering technologists and technicians, except drafters',
 1560.0: 'Surveying and mapping technicians',
 1600.0: 'Agricultural and food scientists',
 1610.0: 'Biological scientists',
 1640.0: 'Conservation scientists and foresters',
 1650.0: 'Other life scientists',
 1660.0: 'Life scientists, all other',
 1700.0: 'Astronomers and physicists',
 1710.0: 'Atmospheric and space scientists',
 1720.0: 'Chemists and materials scientists',
 1745.0: 'Environmental scientists and specialists, including health',
 1750.0: 'Geoscientists and hydrologists, except geographers',
 1760.0: 'Physical scientists, all other',
 1800.0: 'Economists',
 1821.0: 'Clinical and counseling psychologists',
 1822.0: 'School psychologists',
 1825.0: 'Other psychologists',
 1840.0: 'Urban and regional planners',
 1860.0: 'Other social scientists',
 1815.0: 'Survey researchers',
 1830.0: 'Sociologists',
 1900.0: 'Agricultural and food science technicians',
 1910.0: 'Biological technicians',
 1920.0: 'Chemical technicians',
 1935.0: 'Environmental science and geoscience technicians, and nuclear technicians',
 1940.0: 'Nuclear technicians',
 1970.0: 'Other life, physical, and social science technicians',
 1950.0: 'Social science research assistants',
 1980.0: 'Occupational health and safety specialists and technicians',
 2001.0: 'Substance abuse and behavioral disorder counselors',
 2002.0: 'Educational, guidance, and career counselors and advisors',
 2003.0: 'Marriage and family therapists',
 2004.0: 'Mental health counselors',
 2005.0: 'Rehabilitation counselors',
 2006.0: 'Counselors, all other',
 2011.0: 'Child, family, and school social workers',
 2012.0: 'Healthcare social workers',
 2013.0: 'Mental health and substance abuse social workers',
 2014.0: 'Social workers, all other',
 2015.0: 'Probation officers and correctional treatment specialists',
 2016.0: 'Social and human service assistants',
 2025.0: 'Other community and social service specialists',
 2040.0: 'Clergy',
 2050.0: 'Directors, religious activities and education',
 2060.0: 'Religious workers, all other',
 2100.0: 'Lawyers',
 2110.0: 'Judges, magistrates, and other judicial workers',
 2105.0: 'Judicial law clerks',
 2145.0: 'Paralegals and legal assistants',
 2170.0: 'Title examiners, abstractors, and searchers',
 2180.0: 'Legal support workers, all other',
 2205.0: 'Postsecondary teachers',
 2300.0: 'Preschool and kindergarten teachers',
 2310.0: 'Elementary and middle school teachers',
 2320.0: 'Secondary school teachers',
 2330.0: 'Special education teachers',
 2350.0: 'Tutors',
 2360.0: 'Other teachers and instructors',
 2400.0: 'Archivists, curators, and museum technicians',
 2435.0: 'Librarians and media collections specialists',
 2440.0: 'Library technicians',
 2545.0: 'Teaching assistants',
 2555.0: 'Other educational instruction and library workers',
 2600.0: 'Artists and related workers',
 2631.0: 'Commercial and industrial designers',
 2632.0: 'Fashion designers',
 2633.0: 'Floral designers',
 2634.0: 'Graphic designers',
 2635.0: 'Interior designers',
 2636.0: 'Merchandise displayers and window trimmers',
 2640.0: 'Other designers',
 2700.0: 'Actors',
 2710.0: 'Producers and directors',
 2721.0: 'Athletes and sports competitors',
 2722.0: 'Coaches and scouts',
 2723.0: 'Umpires, referees, and other sports officials',
 2740.0: 'Dancers and choreographers',
 2751.0: 'Music directors and composers',
 2752.0: 'Musicians and singers',
 2755.0: 'Disc jockeys, except radio',
 2770.0: 'Entertainers and performers, sports and related workers, all other',
 2805.0: 'Broadcast announcers and radio disc jockeys',
 2810.0: 'News analysts, reporters, and journalists',
 2825.0: 'Public relations specialists',
 2830.0: 'Editors',
 2840.0: 'Technical writers',
 2850.0: 'Writers and authors',
 2861.0: 'Interpreters and translators',
 2862.0: 'Court reporters and simultaneous captioners',
 2865.0: 'Media and communication workers, all other',
 2905.0: 'Broadcast, sound, and lighting technicians',
 2970.0: 'Media and communication equipment workers, all other',
 2910.0: 'Photographers',
 2920.0: 'Television, video, and film camera operators and editors',
 3000.0: 'Chiropractors',
 3010.0: 'Dentists',
 3030.0: 'Dietitians and nutritionists',
 3040.0: 'Optometrists',
 3050.0: 'Pharmacists',
 3065.0: 'Emergency medicine physicians',
 3070.0: ' Radiologists',
 3090.0: 'Other physicians',
 3100.0: 'Surgeons',
 3110.0: 'Physician assistants',
 3120.0: 'Podiatrists',
 3140.0: 'Audiologists',
 3150.0: 'Occupational therapists',
 3160.0: 'Physical therapists',
 3200.0: 'Radiation therapists',
 3210.0: 'Recreational therapists',
 3220.0: 'Respiratory therapists',
 3230.0: 'Speech-language pathologists',
 3235.0: 'Exercise physiologists',
 3245.0: 'Therapists, all other',
 3250.0: 'Veterinarians',
 3255.0: 'Registered nurses',
 3256.0: 'Nurse anesthetists',
 3257.0: 'Nurse midwives',
 3258.0: 'Nurse practitioners',
 3261.0: 'Acupuncturists',
 3270.0: 'Healthcare diagnosing or treating practitioners, all other',
 3300.0: 'Clinical laboratory technologists and technicians',
 3310.0: 'Dental hygienists',
 3321.0: 'Cardiovascular technologists and technicians',
 3322.0: 'Diagnostic medical sonographers',
 3323.0: 'Radiologic technologists and technicians',
 3324.0: 'Magnetic resonance imaging technologists',
 3330.0: 'Nuclear medicine technologists and medical dosimetrists',
 3401.0: 'Emergency medical technicians',
 3402.0: 'Paramedics',
 3421.0: 'Pharmacy technicians',
 3422.0: 'Psychiatric technicians',
 3423.0: 'Surgical technologists',
 3424.0: 'Veterinary technologists and technicians',
 3430.0: 'Dietetic technicians and ophthalmic medical technicians',
 3500.0: 'Licensed practical and licensed vocational nurses',
 3515.0: 'Medical records specialists',
 3520.0: 'Opticians, dispensing',
 3545.0: 'Miscellaneous health technologists and technicians',
 3550.0: 'Other healthcare practitioners and technical occupations',
 3601.0: 'Home health aides',
 3602.0: 'Personal care aides',
 3603.0: 'Nursing assistants',
 3605.0: 'Orderlies and psychiatric aides',
 3610.0: 'Occupational therapy assistants and aides',
 3620.0: 'Physical therapist assistants and aides',
 3630.0: 'Massage therapists',
 3640.0: 'Dental assistants',
 3645.0: 'Medical assistants',
 3646.0: 'Medical transcriptionists',
 3647.0: 'Pharmacy aides',
 3648.0: 'Veterinary assistants and laboratory animal caretakers',
 3649.0: 'Phlebotomists',
 3655.0: 'Other healthcare support workers',
 3700.0: 'First-line supervisors of correctional officers',
 3710.0: 'First-line supervisors of police and detectives',
 3720.0: 'First-line supervisors of firefighting and prevention workers',
 3725.0: ' First-line supervisors of security workers',
 3735.0: 'First-line supervisors of protective service workers, all other',
 3740.0: 'Firefighters',
 3750.0: 'Fire inspectors',
 3801.0: 'Bailiffs',
 3802.0: 'Correctional officers and jailers',
 3820.0: 'Detectives and criminal investigators',
 3830.0: 'Fish and game wardens',
 3840.0: 'Parking enforcement workers',
 3870.0: 'Police officers',
 3900.0: 'Animal control workers',
 3910.0: 'Private detectives and investigators',
 3930.0: 'Security guards and gambling surveillance officers',
 3940.0: 'Crossing guards and flaggers',
 3945.0: 'Transportation security screeners',
 3946.0: 'School bus monitors',
 3960.0: 'Other protective service workers',
 4000.0: 'Chefs and head cooks',
 4010.0: 'First-line supervisors of food preparation and serving workers',
 4020.0: 'Cooks',
 4030.0: 'Food preparation workers',
 4040.0: 'Bartenders',
 4055.0: 'Fast food and counter workers',
 4110.0: 'Waiters and waitresses',
 4120.0: 'Food servers, nonrestaurant',
 4130.0: 'Dining room and cafeteria attendants and bartender helpers',
 4140.0: 'Dishwashers',
 4150.0: 'Hosts and hostesses, restaurant, lounge, and coffee shop',
 4160.0: 'Food preparation and serving related workers, all other',
 4200.0: 'First-line supervisors of housekeeping and janitorial workers',
 4210.0: 'First-line supervisors of landscaping, lawn service, and groundskeeping workers',
 4220.0: 'Janitors and building cleaners',
 4230.0: 'Maids and housekeeping cleaners',
 4240.0: 'Pest control workers',
 4251.0: 'Landscaping and groundskeeping workers',
 4252.0: 'Tree trimmers and pruners',
 4255.0: 'Other grounds maintenance workers',
 4330.0: 'Supervisors of personal care and service workers',
 4340.0: 'Animal trainers',
 4350.0: 'Animal caretakers',
 4400.0: 'Gambling services workers',
 4420.0: 'Ushers, lobby attendants, and ticket takers',
 4435.0: 'Other entertainment attendants and related workers',
 4461.0: 'Embalmers, crematory operators and funeral attendants',
 4465.0: 'Morticians, undertakers, and funeral arrangers',
 4500.0: 'Barbers',
 4510.0: 'Hairdressers, hairstylists, and cosmetologists',
 4521.0: 'Manicurists and pedicurists',
 4522.0: 'Skincare specialists',
 4525.0: 'Other personal appearance workers',
 4530.0: 'Baggage porters, bellhops, and concierges',
 4540.0: 'Tour and travel guides',
 4600.0: 'Childcare workers',
 4621.0: 'Exercise trainers and group fitness instructors',
 4622.0: 'Recreation workers',
 4640.0: 'Residential advisors',
 4655.0: 'Personal care and service workers, all other',
 4700.0: 'First-line supervisors of retail sales workers',
 4710.0: 'First-line supervisors of non-retail sales workers',
 4720.0: 'Cashiers',
 4740.0: 'Counter and rental clerks',
 4750.0: 'Parts salespersons',
 4760.0: 'Retail salespersons',
 4800.0: 'Advertising sales agents',
 4810.0: 'Insurance sales agents',
 4820.0: 'Securities, commodities, and financial services sales agents',
 4830.0: 'Travel agents',
 4840.0: 'Sales representatives of services, except advertising, insurance, financial services, and travel',
 4850.0: 'Sales representatives, wholesale and manufacturing',
 4900.0: 'Models, demonstrators, and product promoters',
 4920.0: 'Real estate brokers and sales agents',
 4930.0: 'Sales engineers',
 4940.0: 'Telemarketers',
 4950.0: 'Door-to-door sales workers, news and street vendors, and related workers',
 4965.0: 'Sales and related workers, all other',
 5000.0: 'First-line supervisors of office and administrative support workers',
 5010.0: 'Switchboard operators, including answering service',
 5020.0: 'Telephone operators',
 5040.0: 'Communications equipment operators, all other',
 5100.0: 'Bill and account collectors',
 5110.0: 'Billing and posting clerks ',
 5120.0: 'Bookkeeping, accounting, and auditing clerks',
 5140.0: 'Payroll and timekeeping clerks',
 5150.0: 'Procurement clerks',
 5160.0: 'Tellers',
 5130.0: ' Gambling cage workers',
 5165.0: 'Financial clerks, all other',
 5220.0: 'Court, municipal, and license clerks',
 5230.0: 'Credit authorizers, checkers, and clerks',
 5240.0: 'Customer service representatives',
 5250.0: 'Eligibility interviewers, government programs',
 5260.0: 'File clerks',
 5300.0: 'Hotel, motel, and resort desk clerks',
 5310.0: 'Interviewers, except eligibility and loan',
 5320.0: 'Library assistants, clerical',
 5330.0: 'Loan interviewers and clerks',
 5340.0: 'New accounts clerks',
 5210.0: 'Correspondence clerks',
 5350.0: 'Order clerks',
 5360.0: 'Human resources assistants, except payroll and timekeeping',
 5400.0: 'Receptionists and information clerks',
 5410.0: 'Reservation and transportation ticket agents and travel clerks',
 5200.0: 'Brokerage clerks',
 5420.0: 'Information and record clerks, all other',
 5500.0: 'Cargo and freight agents',
 5510.0: 'Couriers and messengers',
 5521.0: 'Public safety telecommunicators',
 5522.0: 'Dispatchers, except police, fire, and ambulance',
 5530.0: 'Meter readers, utilities',
 5540.0: 'Postal service clerks',
 5550.0: 'Postal service mail carriers',
 5560.0: 'Postal service mail sorters, processors, and processing machine operators',
 5600.0: 'Production, planning, and expediting clerks',
 5610.0: 'Shipping, receiving, and inventory clerks',
 5630.0: 'Weighers, measurers, checkers, and samplers, recordkeeping',
 5710.0: 'Executive secretaries and executive administrative assistants',
 5720.0: 'Legal secretaries and administrative assistants',
 5730.0: 'Medical secretaries and administrative assistants',
 5740.0: 'Secretaries and administrative assistants, except legal, medical, and executive',
 5810.0: 'Data entry keyers',
 5820.0: 'Word processors and typists',
 5840.0: 'Insurance claims and policy processing clerks',
 5850.0: 'Mail clerks and mail machine operators, except postal service',
 5860.0: 'Office clerks, general',
 5900.0: 'Office machine operators, except computer',
 5910.0: 'Proofreaders and copy markers',
 5920.0: 'Statistical assistants',
 5830.0: 'Desktop publishers',
 5940.0: 'Office and administrative support workers, all other',
 6005.0: 'First-line supervisors of farming, fishing, and forestry workers',
 6010.0: 'Agricultural inspectors',
 6040.0: 'Graders and sorters, agricultural products',
 6020.0: 'Animal breeders',
 6050.0: 'Miscellaneous agricultural workers ',
 6115.0: 'Fishing and hunting workers',
 6120.0: 'Forest and conservation workers',
 6130.0: 'Logging workers',
 6200.0: 'First-line supervisors of construction trades and extraction workers',
 6210.0: 'Boilermakers',
 6220.0: 'Brickmasons, blockmasons, and stonemasons',
 6500.0: 'Reinforcing iron and rebar workers',
 6230.0: 'Carpenters',
 6240.0: 'Carpet, floor, and tile installers and finishers',
 6250.0: 'Cement masons, concrete finishers, and terrazzo workers',
 6260.0: 'Construction laborers',
 6305.0: 'Construction equipment operators',
 6330.0: 'Drywall installers, ceiling tile installers, and tapers',
 6355.0: 'Electricians',
 6360.0: 'Glaziers',
 6400.0: 'Insulation workers',
 6410.0: 'Painters and paperhangers',
 6441.0: 'Pipelayers',
 6442.0: 'Plumbers, pipefitters, and steamfitters',
 6460.0: 'Plasterers and stucco masons',
 6515.0: 'Roofers',
 6520.0: 'Sheet metal workers',
 6530.0: 'Structural iron and steel workers',
 6540.0: 'Solar photovoltaic installers',
 6600.0: 'Helpers, construction trades',
 6660.0: 'Construction and building inspectors',
 6700.0: 'Elevator and escalator installers and repairers',
 6710.0: 'Fence erectors',
 6720.0: 'Hazardous materials removal workers',
 6730.0: 'Highway maintenance workers',
 6740.0: 'Rail-track laying and maintenance equipment operators',
 6765.0: 'Other construction and related workers',
 6750.0: 'Septic tank servicers and sewer pipe cleaners',
 6800.0: 'Derrick, rotary drill, and service unit operators, oil and gas',
 6920.0: 'Roustabouts, oil and gas',
 6825.0: 'Surface mining machine operators and earth drillers',
 6821.0: 'Excavating and loading machine and dragline operators, surface mining',
 6835.0: 'Explosives workers, ordnance handling experts, and blasters',
 6850.0: 'Underground mining machine operators',
 6950.0: 'Other extraction workers',
 7000.0: 'First-line supervisors of mechanics, installers, and repairers',
 7010.0: 'Computer, automated teller, and office machine repairers',
 7020.0: 'Radio and telecommunications equipment installers and repairers',
 7030.0: 'Avionics technicians',
 7040.0: 'Electric motor, power tool, and related repairers',
 7050.0: 'Electrical and electronics installers and repairers, transportation equipment',
 7100.0: 'Electrical and electronics repairers, industrial and utility',
 7110.0: 'Electronic equipment installers and repairers, motor vehicles',
 7120.0: 'Audiovisual equipment installers and repairers',
 7130.0: 'Security and fire alarm systems installers',
 7140.0: 'Aircraft mechanics and service technicians',
 7150.0: 'Automotive body and related repairers',
 7160.0: 'Automotive glass installers and repairers',
 7200.0: 'Automotive service technicians and mechanics',
 7210.0: 'Bus and truck mechanics and diesel engine specialists',
 7220.0: 'Heavy vehicle and mobile equipment service technicians and mechanics',
 7240.0: 'Small engine mechanics',
 7260.0: 'Miscellaneous vehicle and mobile equipment mechanics, installers, and repairers',
 7300.0: 'Control and valve installers and repairers',
 7315.0: 'Heating, air conditioning, and refrigeration mechanics and installers',
 7320.0: 'Home appliance repairers',
 7330.0: 'Industrial and refractory machinery mechanics',
 7340.0: 'Maintenance and repair workers, general',
 7350.0: 'Maintenance workers, machinery',
 7360.0: 'Millwrights',
 7410.0: 'Electrical power-line installers and repairers',
 7420.0: 'Telecommunications line installers and repairers',
 7430.0: 'Precision instrument and equipment repairers',
 7510.0: 'Coin, vending, and amusement machine servicers and repairers',
 7540.0: 'Locksmiths and safe repairers',
 7560.0: 'Riggers',
 7610.0: 'Helpers--installation, maintenance, and repair workers ',
 7440.0: 'Wind turbine service technicians',
 7520.0: 'Commercial divers',
 7550.0: 'Manufactured building and mobile home installers',
 7640.0: 'Other installation, maintenance, and repair workers',
 7700.0: 'First-line supervisors of production and operating workers',
 7720.0: 'Electrical, electronics, and electromechanical assemblers',
 7730.0: 'Engine and other machine assemblers',
 7740.0: 'Structural metal fabricators and fitters',
 7710.0: 'Aircraft structure, surfaces, rigging, and systems assemblers',
 7750.0: 'Other assemblers and fabricators',
 7800.0: 'Bakers',
 7810.0: 'Butchers and other meat, poultry, and fish processing workers',
 7830.0: 'Food and tobacco roasting, baking, and drying machine operators and tenders',
 7840.0: 'Food batchmakers',
 7850.0: 'Food cooking machine operators and tenders',
 7855.0: 'Food processing workers, all other',
 7905.0: 'Computer numerically controlled tool operators and programmers',
 7925.0: 'Forming machine setters, operators, and tenders, metal and plastic',
 7950.0: 'Cutting, punching, and press machine setters, operators, and tenders, metal and plastic',
 8000.0: 'Grinding, lapping, polishing, and buffing machine tool setters, operators, and tenders, metal and plastic',
 8025.0: 'Other machine tool setters, operators, and tenders, metal and plastic',
 8030.0: 'Machinists',
 8040.0: 'Metal furnace operators, tenders, pourers, and casters',
 8060.0: 'Model makers and patternmakers, metal and plastic',
 8100.0: 'Molders and molding machine setters, operators, and tenders, metal and plastic',
 8130.0: 'Tool and die makers',
 8140.0: 'Welding, soldering, and brazing workers',
 8225.0: 'Other metal workers and plastic workers',
 8250.0: 'Prepress technicians and workers',
 8255.0: 'Printing press operators',
 8256.0: 'Print binding and finishing workers',
 8300.0: 'Laundry and dry-cleaning workers',
 8310.0: 'Pressers, textile, garment, and related materials',
 8320.0: 'Sewing machine operators',
 8335.0: 'Shoe and leather workers',
 8350.0: 'Tailors, dressmakers, and sewers',
 8365.0: 'Textile machine setters, operators, and tenders',
 8450.0: 'Upholsterers',
 8465.0: 'Other textile, apparel, and furnishings workers',
 8500.0: 'Cabinetmakers and bench carpenters',
 8510.0: 'Furniture finishers',
 8530.0: 'Sawing machine setters, operators, and tenders, wood',
 8540.0: 'Woodworking machine setters, operators, and tenders, except sawing',
 8555.0: 'Other woodworkers',
 8600.0: 'Power plant operators, distributors, and dispatchers',
 8610.0: 'Stationary engineers and boiler operators',
 8620.0: 'Water and wastewater treatment plant and system operators',
 8630.0: 'Miscellaneous plant and system operators',
 8640.0: 'Chemical processing machine setters, operators, and tenders',
 8650.0: 'Crushing, grinding, polishing, mixing, and blending workers',
 8710.0: 'Cutting workers',
 8720.0: 'Extruding, forming, pressing, and compacting machine setters, operators, and tenders',
 8730.0: 'Furnace, kiln, oven, drier, and kettle operators and tenders',
 8740.0: 'Inspectors, testers, sorters, samplers, and weighers',
 8750.0: 'Jewelers and precious stone and metal workers',
 8760.0: 'Dental and ophthalmic laboratory technicians and medical appliance technicians',
 8800.0: 'Packaging and filling machine operators and tenders',
 8810.0: 'Painting workers',
 8830.0: 'Photographic process workers and processing machine operators',
 8850.0: 'Adhesive bonding machine operators and tenders',
 8910.0: 'Etchers and engravers',
 8920.0: 'Molders, shapers, and casters, except metal and plastic',
 8930.0: 'Paper goods machine setters, operators, and tenders',
 8940.0: 'Tire builders',
 8950.0: 'Helpers--production workers',
 8865.0: 'Other production equipment operators and tenders',
 8990.0: 'Other production workers',
 9005.0: 'Supervisors of transportation and material moving workers',
 9030.0: 'Aircraft pilots and flight engineers',
 9040.0: 'Air traffic controllers and airfield operations specialists',
 9050.0: 'Flight attendants',
 9110.0: 'Ambulance drivers and attendants, except emergency medical technicians',
 9121.0: 'Bus drivers, school',
 9122.0: 'Bus drivers, transit and intercity',
 9130.0: 'Driver/sales workers and truck drivers',
 9141.0: 'Shuttle drivers and chauffeurs',
 9142.0: 'Taxi drivers',
 9150.0: 'Motor vehicle operators, all other',
 9210.0: 'Locomotive engineers and operators',
 9240.0: 'Railroad conductors and yardmasters',
 9265.0: 'Other rail transportation workers',
 9300.0: 'Sailors and marine oilers',
 9330.0: 'Ship engineers',
 9310.0: 'Ship and boat captains and operators',
 9350.0: 'Parking attendants',
 9365.0: 'Transportation service attendants',
 9410.0: 'Transportation inspectors',
 9415.0: 'Passenger attendants',
 9430.0: 'Other transportation workers',
 9510.0: 'Crane and tower operators',
 9570.0: 'Conveyor, dredge, and hoist and winch operators',
 9600.0: 'Industrial truck and tractor operators',
 9610.0: 'Cleaners of vehicles and equipment',
 9620.0: 'Laborers and freight, stock, and material movers, hand',
 9630.0: 'Machine feeders and offbearers',
 9640.0: 'Packers and packagers, hand',
 9645.0: 'Stockers and order fillers',
 9650.0: 'Pumping station operators',
 9720.0: 'Refuse and recyclable material collectors',
 9760.0: 'Other material moving workers',
 9800.0: 'Military officer special and tactical operations leaders',
 9810.0: 'First-line enlisted military supervisors',
 9825.0: 'Military enlisted tactical operations and air/weapons specialists and crew members',
 9830.0: 'Military, rank not specified',
 9920.0: 'Unemployed, with no work experience in the last 5 years or earlier or never worked'})

income.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,observed
1,30 - 50,Employee of a private for-profit company or bu...,Bachelor’s degree,Never married or under 15 years old,Medical and health services managers,Georgia/GA,Reference person,40 - 50,Female,White alone,0
2,30 - 50,Employee of a private for-profit company or bu...,GED or alternative credential,Married,Construction laborers,Georgia/GA,Reference person,40 - 50,Male,White alone,1
5,30 - 50,State government employee,Regular high school diploma,Divorced,Education and childcare administrators,Alabama/AL,Reference person,20 - 30,Female,Black or African American alone,0
10,30 - 50,"Self-employed in own incorporated business, pr...",Regular high school diploma,Married,"Welding, soldering, and brazing workers",Mexico,Reference person,50 - 60,Male,White alone,1
11,50 - 70,Employee of a private for-profit company or bu...,Associate’s degree,Married,Human resources managers,Alabama/AL,Husband/wife,30 - 40,Female,Black or African American alone,1


In [24]:
# save to file
income.to_csv('folktables_adult_2017.csv', index = False) #change name according to year downloaded